# NetworkX - beagle Clustering
------------------

- Try to get KarateClub working.
- Getting correlation coefficitions / centrality measures for nodes in ancestor chain

# Preparation

In [1]:
import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.linear_model.logistic import LogisticRegression
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = 500

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df_zeek = pd.read_json("combined_zeek.log", lines = True)
df_zeek.replace('nan', np.nan, inplace=True)
print(type(df_zeek))
print(df_zeek.shape)
#print(df_zeek.EventID.value_counts())
df_zeek.head()

<class 'pandas.core.frame.DataFrame'>
(2140, 148)


,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conn,bobs.bigwheel.local,zeek,1.588205e+09,CvxbPE3MuO7boUdSc8,10.0.1.6,138.0,10.0.1.255,138.0,udp,NaN,NaN,NaN,NaN,S0,0.0,D,1.0,229.0,0.0,0.0,00:0d:3a:13:e3:69,ff:ff:ff:ff:ff:ff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [3]:
df_label = pd.read_json('./df_labelled_coalesce.json', lines=True)
df_label['TP_Binary'] = df_label.TP_Label!='0'
df_label.replace('-', np.nan, inplace=True)
print(type(df_label))
print(df_label.shape)
print(df_label.EventID.value_counts())
df_label.head(1)

<class 'pandas.core.frame.DataFrame'>
(198760, 390)
12       61158
10       39286
7        20259
13       17542
4658     10973
         ...  
172          1
46           1
50103        1
306          1
41           1
Name: EventID, Length: 165, dtype: int64


,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,SubLayerKey,SubLayerName,SubLayerType,Default SD String:,ClassId,ClassName,CompatibleIds,DeviceDescription,DeviceId,LocationInformation,VendorIds,TargetProcessName,SecurityPackageName,AdditionalInfo,AdditionalInfo2,ProviderType,RuleAttr,AdapterName,AdapterSuffixName,DnsServerList,ErrorCode,Ipaddress,Sent UpdateServer,TaskContentNew,TaskName,ActiveProfile,MiniportName,MiniportNameLen,GroupPolicyApplied,LogDroppedPacketsEnabled,LogSuccessfulConnectionsEnabled,MulticastFlowsEnabled,OperationMode,Profile,RemoteAdminEnabled,TargetInfo,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,

In [4]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
#    In [9]: result = pd.concat([df1, df4], axis=1, sort=False)
df_label = df_label.append(df_zeek, ignore_index=True, sort=False)

In [5]:
df_label.shape

(200900, 536)

In [6]:
df_label.head(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Gr

In [7]:
df_label.tail(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Gr

## Normalize Timestamp, IP/Port Columns
------------------

In [8]:
df_label['time_stamp'] = df_label['@timestamp'].apply(lambda x: (pd.Timestamp(x)))

In [9]:
df_label['time_stamp'] = df_label['time_stamp'].fillna(df_label['ts'].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True))))

In [10]:
# Transform Host Data - Similar Data to Single Columns
df_label['Destination_IP'] = df_label['DestinationIp']
df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['DestAddress'])
df_label['Source_IP'] = df_label['IpAddress']
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceIp'])
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceAddress'])
df_label['Source_Port'] = df_label['IpPort']
df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['SourcePort'])
df_label['Destination_Port'] = df_label['DestinationPort']
df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['DestPort'])

In [11]:
# Transform Network Data - Similar Data to Single Columns
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['id_orig_h'])
df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['id_orig_p'])
df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['id_resp_h'])
df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['id_resp_p'])

## Beagle Column Formatting

In [12]:
# Sysmon Transformer
df_label['EventData_ParentImage'] = df_label['ParentImage'].astype(object)
df_label['EventData_ParentProcessId'] = df_label['ParentProcessId']
df_label['EventData_ParentProcessGuid'] = df_label['ParentProcessGuid']
df_label['EventData_Image'] = df_label['Image'].astype(object)
df_label['EventData_CommandLine'] = df_label['CommandLine'].astype(object)
df_label['EventData_Hashes'] = df_label['Hashes'].astype(object)

df_label['Computer'] = df_label['Hostname'].astype(object)
df_label['EventData_User'] = df_label['User'].astype(object)
df_label['EventData_UtcTime'] = df_label['UtcTime']

df_label['EventData_ProcessGuid'] = df_label['ProcessGuid']
df_label['EventData_ProcessId'] = df_label['ProcessId']

df_label['EventData_DestinationIp'] = df_label['DestinationIp']
df_label['EventData_DestinationPort'] = df_label['DestinationPort']
df_label['EventData_Protocol'] = df_label['Protocol']
df_label['EventData_TargetFilename'] = df_label['TargetFilename']

df_label['EventData_TargetObject'] = df_label['TargetObject']
df_label['EventData_EventType'] = df_label['EventType']
#df_label['EventData_Details'] = df_label['ProcessId']

# EVTX Transformer
#event_id = int(event["eventid_qualifiers"])
df_label['eventid_qualifiers'] = df_label['EventID']

#parent_pid = int(event["data_name_processid"], 16)
df_label['data_name_processid'] = df_label['ProcessId']

#child_pid = int(event["data_name_newprocessid"], 16)
df_label['data_name_newprocessid'] = df_label['NewProcessId']

#proc_name, proc_path = split_path(event["data_name_newprocessname"])
df_label['data_name_newprocessname'] = df_label['NewProcessName']

#child = Process(
#    host=event["computer"],
df_label['computer'] = df_label['Hostname'].astype(object)
#    process_id=child_pid,
#    user=event["data_name_subjectusername"],
df_label['data_name_subjectusername'] = df_label['SubjectUserName']

#    process_image=proc_name,
#    process_image_path=proc_path,
#    command_line=event.get("data_name_commandline"),
df_label['data_name_commandline'] = df_label['CommandLine']
#)
#parent.launched[child].append(timestamp=event["timecreated_systemtime"])
df_label['timecreated_systemtime'] = df_label['EventTime']


In [13]:
a = df_label.loc[df_label['EventID']==4688].head(1)
a = a[a.columns[~a.isnull().all()]]
a.drop(columns='Message')

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,Keywords,Opcode,OpcodeValue,ProviderGuid,RecordNumber,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Task,ThreadID,Version,host,port,tags,TP_Label,ProcessId,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,TP_Binary,time_stamp,EventData_CommandLine,Computer,EventData_ProcessId,EventData_EventType,eventid_qualifiers,data_name_processid,data_name_newprocessid,data_name_newprocessname,computer,data_name_subjectusername,data_name_commandline,timecreated_systemtime
40,2020-05-02T03:06:14.178Z,1.0,Process Creation,Security,4688.0,2020-05-01 23:06:14,2020-05-01 23:06:13,AUDIT_SUCCESS,4.0,NEWYORK.dmevals.local,-9.214365e+18,Info,0.0,{54849625-5478-4994-A5BA-3E3B0328C30D},151599.0,INFO,2.0,eventlog,im_msvistalog,Microsoft-Windows-Security-Auditing,13312.0,288.0,2.0,wec.internal.cloudapp.net,60737.0,[mordorDataset],0,0x788,DMEVALS,0x3e7,NEWYORK$,S-1-5-18,C:\windows\system32\usoclient.exe StartScan,S-1-16-16384,0x6f8,C:\Windows\System32\UsoClient.exe,C:\Windows\System32\svchost.exe,0x0,S-1-0-0,%%1936,False,2020-05-02 03:06:14.178000+00:00,C:\windows\system32\usoclient.exe StartScan,NEWYORK.dmevals.local,0x788,AUDIT_SUCCESS,4688.0,0x788,0x6f8,C:\Windows\System32\UsoClient.exe,NEWYORK.dmevals.local,NEWYORK$,C:\windows\system32\usoclient.exe StartScan,2020-05-01 23:06:13


In [14]:
import os
import json
from typing import Dict, Generator, List

from beagle.datasources.base_datasource import DataSource
from beagle.transformers import GenericTransformer
from beagle.transformers.evtx_transformer import WinEVTXTransformer

class WinJSONData(DataSource):
    """A generic data source which returns events one by one
    """

    name = "JSON Data"
    transformers = [WinEVTXTransformer]
    category = "Generic Data"

    def __init__(self, events: List[Dict]) -> None:
        self._events = events

    def events(self) -> Generator[dict, None, None]:
        for event in self._events:
            yield event

    def metadata(self) -> dict:
        return {}

In [15]:
#from beagle.backends import NetworkX

#NX = NetworkX.from_datasources(
#    datasources=[
#        WinJSONData(df_label.loc[(df_label['EventID']==4688)].head(100).to_dict('records')),
#    ]
#)
#G = NX.graph()

## Manual Recurse Ancestor Chain Functions

In [16]:
def eval_ancestor_chain(G, starting_guid):
    for (p, d) in G.nodes(data=True):
        if 'process_guid' in d['properties'].keys():
            if starting_guid in d['properties']['process_guid']:
                selected_node = p
                
    #ancestors = nx.ancestors(G, selected_node)
    malicious_ancestors = [selected_node]
    for i in nx.ancestors(G, selected_node):
        if(G.node[i]['_node_type'] == 'Process'):
            print("==========================================")
            print("Node Data:")
            print("==========================================")
            print(G.node[i])
            print("==========================================")
            answer = None
            while answer not in ("yes", "no"):
                answer = input("Malicious? Enter yes or no: ")
                if answer == "yes":
                    # Do this.
                    malicious_ancestors.append(i)
                elif answer == "no":
                    # Do that.
                    return malicious_ancestors
                else:
                    print("Please enter yes or no.")
    return malicious_ancestors

def determine_malicious_root_proc(G, starting_guid):
    malicious_ancestors = eval_ancestor_chain(G, starting_guid)
    print("")
    print("##########################################")
    print("Confirm this as the senior malicious node?")
    print("##########################################")
    print("Node Data:")
    print("==========================================")
    print(G.node[malicious_ancestors[-1]])
    print("==========================================")
    print("Edge Out Data:")
    print("==========================================")
    for i in G.out_edges(malicious_ancestors[-1], data=True):
        if i[2]['type'] == 'Launched':
            print(i[2]['type'])
            print(G.node[i[1]])
            print("==========================================")
    answer = None
    while answer not in ("yes", "no"):
        answer = input("Malicious? Enter yes or no: ")
        if answer == "yes":
            # Do this.
            return malicious_ancestors[-1]
        elif answer == "no":
            # Do that.
            print("Re-run the eval_ancestor chain")
            malicious_root_ancestor = determine_malicious_root_proc(G, starting_guid)
            return malicious_root_ancestor
        else:
            print("Please enter yes or no.")
            
def get_ancestor_descendant_graph(G, malicious_root_ancestor):
    print(malicious_root_ancestor)

    descendants = nx.descendants(G, malicious_root_ancestor) #Returns all nodes reachable from source in G.
    print(len(descendants))

    ancestors = nx.ancestors(G, malicious_root_ancestor)
    print(len(ancestors))

    suspected_malicious = list(ancestors) + list(descendants)
    suspected_malicious.append(malicious_root_ancestor)
    print(len(suspected_malicious))

    H = G.subgraph(suspected_malicious)
    return H, malicious_root_ancestor, ancestors, descendants

def print_ancestor_descendant_graph(H, malicious_root_ancestor, ancestors, descendants):
    from matplotlib.pyplot import figure
    # Why are there orphaned nodes? I forgot to add the OG node. Fixed.
    # https://networkx.github.io/documentation/networkx-1.9/examples/drawing/labels_and_colors.html
    #fig = plt.Figure()
    figure(figsize=(15, 12))
    pos=nx.kamada_kawai_layout(H) # positions for all nodes

    # nodes
    nx.draw_networkx_nodes(H,pos,
                       nodelist=list(ancestors),
                       node_color='r',
                       #node_size=500,
                       label='ancestors',
                   alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                       nodelist=list(descendants),
                       node_color='b',
                       #node_size=500,
                       label='descendants',
                   alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                       nodelist=[malicious_root_ancestor],
                       node_color='g',
                       #node_size=500,
                       label='starting-point',
                   alpha=0.8)

    # edges
    nx.draw_networkx_edges(H,pos,width=1.0,alpha=0.5)

    #plt.legend()
    plt.legend(loc='upper left')
    #fig.update_layout(legend=dict(x=-.1, y=1.2))
    plt.axis('off')
    print("Saving as: ancestor_descendants_" + str(malicious_root_ancestor) + ".png")
    plt.savefig("ancestor_descendants_" + str(malicious_root_ancestor) + ".png") # save as png
    plt.show() # display
    
def get_ancestor_descendant_types(H):
    file_nodes = []
    for (p, d) in H.nodes(data=True):
        if d['_node_type']=='File':
            file_nodes.append(p)
    len(file_nodes)

    proc_nodes = []
    for (p, d) in H.nodes(data=True):
        if d['_node_type']=='Process':
            proc_nodes.append(p)
    len(proc_nodes)

    ip_nodes = []
    for (p, d) in H.nodes(data=True):
        if d['_node_type']=='IP Address':
            ip_nodes.append(p)
    len(ip_nodes)
    
    return file_nodes, proc_nodes, ip_nodes

def print_ancestor_descendant_type_graph(H, malicious_root_ancestor, file_nodes, proc_nodes, ip_nodes):
    # Why are there orphaned nodes? I forgot to add the OG node. Fixed.
    # https://networkx.github.io/documentation/networkx-1.9/examples/drawing/labels_and_colors.html
    #fig = plt.Figure()
    figure(figsize=(15, 12))
    pos=nx.kamada_kawai_layout(H) # positions for all nodes

    # nodes
    nx.draw_networkx_nodes(H,pos,
                           nodelist=file_nodes,
                           node_color='r',
                           #node_size=500,
                           label='file_nodes',
                       alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                           nodelist=proc_nodes,
                           node_color='b',
                           #node_size=500,
                           label='proc_nodes',
                       alpha=0.8)
    nx.draw_networkx_nodes(H,pos,
                           nodelist=ip_nodes,
                           node_color='g',
                           #node_size=500,
                           label='ip_nodes',
                       alpha=0.8)

    # edges
    nx.draw_networkx_edges(H,pos,width=1.0,alpha=0.5)

    #plt.legend()
    plt.legend(loc='upper left')
    #fig.update_layout(legend=dict(x=-.1, y=1.2))
    plt.axis('off')
    print("Saving as: ancestor_descendants_nodeType_" + str(malicious_root_ancestor) + ".png")
    plt.savefig("ancestor_descendants_nodeType_" + str(malicious_root_ancestor) + ".png") # save as png
    plt.show() # display
    
def analyze_graph_vs_label_proc(H, proc_nodes):
    graph_bad_guids = []
    for i in proc_nodes:
        graph_bad_guids.append(H.node[i]['properties']['process_guid'])
    print(len(graph_bad_guids))
    #print(graph_bad_guids)
    
    label_bad_guids = df_label.loc[(df_label['TP_Binary'] == True) & (df_label.ProcessGuid.notnull())]['ProcessGuid'].unique()
    pd_all_guids = df_label.loc[(df_label.ProcessGuid.notnull()) & (df_label.CommandLine.notnull())][['ProcessGuid', 'Hostname', 'TP_Binary', 'CommandLine']]
    pd_all_guids['orig_label'] = False
    pd_all_guids['orig_label'].loc[pd_all_guids.ProcessGuid.isin(label_bad_guids)] = True

    pd_all_guids = pd_all_guids.drop(columns=['TP_Binary']).drop_duplicates()
    pd_all_guids['graph_label'] = False
    pd_all_guids['graph_label'].loc[pd_all_guids.ProcessGuid.isin(graph_bad_guids)] = True

    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
        display(pd_all_guids[['ProcessGuid', 'Hostname', 'orig_label', 'graph_label', 'CommandLine']].loc[((pd_all_guids.orig_label==True) | (pd_all_guids.graph_label==True))].sort_values(by=['orig_label', 'graph_label'], ascending=False))
        
def manual_recurse_pipeline(G, starting_point):
    malicious_root_ancestor = determine_malicious_root_proc(G, starting_point)
    print(malicious_root_ancestor)
    
    H, malicious_root_ancestor, ancestors, descendants = get_ancestor_descendant_graph(G, malicious_root_ancestor)
    
    #print_ancestor_descendant_graph(H, malicious_root_ancestor, ancestors, descendants)
    
    file_nodes, proc_nodes, ip_nodes = get_ancestor_descendant_types(H)
    
    #print_ancestor_descendant_type_graph(H, malicious_root_ancestor, file_nodes, proc_nodes, ip_nodes)
    
    #analyze_graph_vs_label_proc(H, proc_nodes)
    
    return H, malicious_root_ancestor, ancestors, descendants, file_nodes, proc_nodes, ip_nodes


# Testing Beagle

In [17]:
from beagle.backends import NetworkX
from beagle.transformers import SysmonTransformer

In [18]:
import os
import json
from typing import Dict, Generator, List

from beagle.datasources.base_datasource import DataSource
from beagle.transformers import GenericTransformer
from beagle.transformers.sysmon_transformer import SysmonTransformer

class JSONData(DataSource):
    """A generic data source which returns events one by one
    """

    name = "JSON Data"
    transformers = [SysmonTransformer]
    category = "Generic Data"

    def __init__(self, events: List[Dict]) -> None:
        self._events = events

    def events(self) -> Generator[dict, None, None]:
        for event in self._events:
            yield event

    def metadata(self) -> dict:
        return {}

In [19]:
from beagle.backends import NetworkX

# 4688 events just throw things off with no added information.
NX = NetworkX.from_datasources(
    datasources=[
#        WinJSONData(df_label.loc[(df_label['EventID']==4688)].to_dict('records')),
        JSONData(df_label.loc[(df_label['EventID']==1)].to_dict('records')),
        JSONData(df_label.loc[(df_label['EventID']==3)].to_dict('records')),
        JSONData(df_label.loc[(df_label['EventID']==11)].to_dict('records')),
    ]
)
G = NX.graph()

2020-08-06T10:47:02 | beagle.transformers.sysmon_transformer.__init__:15 | INFO | Created Sysmon Transformer.
2020-08-06T10:47:02 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-08-06T10:47:02 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-08-06T10:47:02 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-08-06T10:47:02 | beagle.transformers.base_transformer._consumer_thread:143 | WARNING | Error when parsing event, recieved exception 'float' object has no attribute 'split'
2020-08-06T10:47:02 | beagle.transformers.base_transformer.run:111 | INFO | Finished processing of events, created 1784 nodes.
2020-08-06T10:47:02 | beagle.transformers.base_transfor

In [20]:
G

In [21]:
# KeyError: 'edge_name' -> first time hitting this. Strange.
graph_json = NetworkX.graph_to_json(G)

2020-08-06T10:47:03 | beagle.backends.networkx.__init__:56 | INFO | Initialized NetworkX Backend


In [22]:
from networkx.readwrite import json_graph

G = json_graph.node_link_graph(graph_json)

In [23]:
G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute='Orig_Node_ID')

In [24]:
# Some of the 4866 events just get no data besides Process ID I guess...
pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

,properties,_node_type,_node_class,_display,_color,Orig_Node_ID
0,"{'process_guid': '{47ab858c-cadc-5eac-2200-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 1448, 'user': None, 'process_image': 'svchost.exe', 'process_image_path': 'C:\Windows\System32', 'command_line': None, 'hashes': {}, 'process_path': 'C:\Windows\System32\svchost.exe'}",Process,SysMonProc,svchost.exe,#FF0000,-290913363542393754
1,"{'process_guid': '{47ab858c-e589-5eac-f803-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 8288, 'user': 'DMEVALS\pbeesly', 'process_image': 'taskhostw.exe', 'process_image_path': 'C:\Windows\System32', 'command_line': 'taskhostw.exe Install $(Arg0)', 'hashes': {'sha1': '9245F36AA783167E0AFE948B1C0E8E7F3B81E3D4', 'md5': '8AC325C757FA721B272ECEA19EBCF745', 'sha256': '0BEC3128B241FA533DC5A9D3F62AA41323D205DEFC30DCDE228C5ECC2D1E50A9', 'imphash': '9CB27CAED52CB0AFFB32788922A0D083...",Process,SysMonProc,taskhostw.exe,#FF0000,4937566340489533370
2,"{'process_guid': '{47ab858c-e296-5eac-c803-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 760, 'user': 'NT AUTHORITY\NETWORK SERVICE', 'process_image': 'gpupdate.exe', 'process_image_path': 'C:\Windows\System32', 'command_line': 'gpupdate.exe /target:computer', 'hashes': {'sha1': '1529CB7F7DFA798337F781A71B583371E1D43F2B', 'md5': 'D4F01BEC9CA921C13ED1BDD1BF1D2D24', 'sha256': 'FE428F64B6920CBD542BF7097F009A576673888967CB5AE8803D310667ED428D', 'imphash': '874ADC3991B152C2F8C8A...",Process,SysMonProc,gpupdate.exe,#FF0000,-1505689213524721999
3,"{'process_guid': '{47ab858c-e2d4-5eac-cd03-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 8532, 'user': 'DMEVALS\pbeesly', 'process_image': 'taskhostw.exe', 'process_image_path': 'C:\Windows\System32', 'command_line': 'taskhostw.exe Install $(Arg0)', 'hashes': {'sha1': '9245F36AA783167E0AFE948B1C0E8E7F3B81E3D4', 'md5': '8AC325C757FA721B272ECEA19EBCF745', 'sha256': '0BEC3128B241FA533DC5A9D3F62AA41323D205DEFC30DCDE228C5ECC2D1E50A9', 'imphash': '9CB27CAED52CB0AFFB32788922A0D083...",Process,SysMonProc,taskhostw.exe,#FF0000,2581757294000610704
4,"{'process_guid': '{47ab858c-e37a-5eac-e003-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 1548, 'user': 'DMEVALS\pbeesly', 'process_image': 'taskhostw.exe', 'process_image_path': 'C:\Windows\System32', 'command_line': 'taskhostw.exe Event', 'hashes': {'sha1': '9245F36AA783167E0AFE948B1C0E8E7F3B81E3D4', 'md5': '8AC325C757FA721B272ECEA19EBCF745', 'sha256': '0BEC3128B241FA533DC5A9D3F62AA41323D205DEFC30DCDE228C5ECC2D1E50A9', 'imphash': '9CB27CAED52CB0AFFB32788922A0D083'}, 'proce...",Process,SysMonProc,taskhostw.exe,#FF0000,3595427489194071912
...,...,...,...,...,...,...
2049,"{'process_guid': '{5aa8ec29-cae0-5eac-5100-000000000400}', 'host': 'NASHUA.dmevals.local', 'process_id': 3428, 'user': nan, 'process_image': 'Sysmon.exe', 'process_image_path': 'C:\windows', 'command_line': None, 'hashes': {}, 'process_path': 'C:\windows\Sysmon.exe'}",Process,SysMonProc,Sysmon.exe,#FF0000,2619389006651179066
2050,"{'host': None, 'file_path': 'C:\Windows\System32\config\systemprofile\AppData\Roaming\Microsoft\SystemCertificates\My\Certificates', 'file_name': 'A4F69156AB2E4C5AFBB55BFD66DC60F0D398FE9A', 'full_path': 'C:\Windows\System32\config\systemprofile\AppData\Roaming\Microsoft\SystemCertificates\My\Certificates\A4F69156AB2E4C5AFBB55BFD66DC60F0D398FE9A', 'extension': None, 'hashes': {}}",File,File,A4F69156AB2E4C5AFBB55BFD66DC60F0D398FE9A,#3CB371,7734894780514096052
2051,"{'host': 'NASHUA.dmevals.local', 'file_path': 'C:\windows', 'file_name': 'Sysmon.exe', 'full_path': 'C:\windows\Sysmon.exe', 'extension': None, 'hashes': {}}",File,File,Sysmon.exe,#3CB371,6488379901365441725
2052,"{'host': 'SCRANTON.dmevals.local', 'file_path': 'C:\Program Files\SysinternalsSuite', 'file_name': 'accesschk.exe', 'full_path': 'C:\Program Files\SysinternalsSuite\accesschk.exe', 'extension': None, 'hashes': {}}",File,File,accesschk.exe,#3CB371,-986666193450779071


# Start from the bottom - Scranton

## Malicious Subgraph

In [25]:
# PSEXEC on SCRANTON - '{47ab858c-e558-5eac-f503-000000000400}'
M2, malicious_root_ancestor, ancestors, descendants, file_nodes, proc_nodes, ip_nodes = manual_recurse_pipeline(G, '{47ab858c-e558-5eac-f503-000000000400}')

Node Data:
{'properties': {'process_guid': '{47ab858c-e13c-5eac-a903-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 8524, 'user': nan, 'process_image': 'â€®cod.3aka3.scr', 'process_image_path': 'C:\\ProgramData\\victim', 'command_line': '"C:\\ProgramData\\victim\\â€®cod.3aka3.scr" /S', 'hashes': {'sha1': '4B7FA56A4E85F88B98D11A6E018698AE3FBA5E62', 'md5': '9D1C5EF38E6073661C74660B3A71A76E', 'sha256': '0DF38A55D940F498478EB03683C94D4584236E100125B526A67650BA54DF4AE4', 'imphash': 'F00447512A354E59D39D2818AABA4A17'}, 'process_path': 'C:\\ProgramData\\victim\\â€®cod.3aka3.scr'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'â€®cod.3aka3.scr', '_color': '#FF0000', 'Orig_Node_ID': 691507765666240315}


Malicious? Enter yes or no:  yes


Node Data:
{'properties': {'process_guid': '{47ab858c-e374-5eac-d803-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 3852, 'user': nan, 'process_image': 'powershell.exe', 'process_image_path': 'C:\\windows\\system32\\WindowsPowerShell\\v1.0', 'command_line': 'powershell.exe', 'hashes': {'sha1': '36C5D12033B2EAF251BAE61C00690FFB17FDDC87', 'md5': 'CDA48FC75952AD12D99E526D0B6BF70A', 'sha256': '908B64B1971A979C7E3E8CE4621945CBA84854CB98D76367B791A6E22B5F6D53', 'imphash': 'A7CEFACDDA74B13CD330390769752481'}, 'process_path': 'C:\\windows\\system32\\WindowsPowerShell\\v1.0\\powershell.exe'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'powershell.exe', '_color': '#FF0000', 'Orig_Node_ID': -8283985431713753684}


Malicious? Enter yes or no:  yes


Node Data:
{'properties': {'process_guid': '{47ab858c-e1e4-5eac-b803-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 2976, 'user': nan, 'process_image': 'PowerShell.exe', 'process_image_path': 'C:\\windows\\system32\\WindowsPowerShell\\v1.0', 'command_line': '"PowerShell.exe" -noni -noexit -ep bypass -window hidden -c "sal a New-Object;Add-Type -AssemblyName \'System.Drawing\'; $g=a System.Drawing.Bitmap(\'C:\\Users\\pbeesly\\Downloads\\monkey.png\');$o=a Byte[] 4480;for($i=0; $i -le 6; $i++){foreach($x in(0..639)){$p=$g.GetPixel($x,$i);$o[$i*640+$x]=([math]::Floor(($p.B-band15)*16)-bor($p.G-band15))}};$g.Dispose();IEX([System.Text.Encoding]::ASCII.GetString($o[0..3932]))"', 'hashes': {'sha1': '36C5D12033B2EAF251BAE61C00690FFB17FDDC87', 'md5': 'CDA48FC75952AD12D99E526D0B6BF70A', 'sha256': '908B64B1971A979C7E3E8CE4621945CBA84854CB98D76367B791A6E22B5F6D53', 'imphash': 'A7CEFACDDA74B13CD330390769752481'}, 'process_path': 'C:\\windows\\system32\\WindowsPowerShell\\v1.0\\Pow

...

Node Data:
{'properties': {'process_guid': '{47ab858c-dac4-5eac-f202-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 4440, 'user': nan, 'process_image': 'Explorer.EXE', 'process_image_path': 'C:\\windows', 'command_line': None, 'hashes': {}, 'process_path': 'C:\\windows\\Explorer.EXE'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'Explorer.EXE', '_color': '#FF0000', 'Orig_Node_ID': 2141826236717757093}


Malicious? Enter yes or no:  no



##########################################
Confirm this as the senior malicious node?
##########################################
Node Data:
{'properties': {'process_guid': '{47ab858c-e1e3-5eac-b603-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 4892, 'user': 'DMEVALS\\pbeesly', 'process_image': 'control.exe', 'process_image_path': 'C:\\Windows\\System32', 'command_line': '"C:\\Windows\\System32\\control.exe"  /name Microsoft.BackupAndRestoreCenter', 'hashes': {'sha1': 'D054A1D1E0BECCA5EEF751CF616ECB811CFABECE', 'md5': '62D970D8B60F75C12D21C740F2D8A5DA', 'sha256': 'D6E21DA3BE0701162A36F8C9F94E616B1A0C5FD4CC1B52EFD81959CB257957C1', 'imphash': '7A8EC2645C24D85DE8216D63022623C0'}, 'process_path': 'C:\\Windows\\System32\\control.exe'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'control.exe', '_color': '#FF0000', 'Orig_Node_ID': 7742364843485053848}
Edge Out Data:
Launched
{'properties': {'process_guid': '{47ab858c-e1e4-5eac-b803-000000000400}', 'hos

Malicious? Enter yes or no:  yes


1076
1076
468
10
479


## Clustering

In [26]:
# No longer needed.
#H = G.subgraph(max(nx.weakly_connected_components(G), key=len))
#H

In [27]:
# Degree Centrality
# Works on G
c_degree = nx.degree_centrality(G)
nx.set_node_attributes(G, c_degree, 'DegreeCentrality')
len(c_degree)

2054

In [28]:
# Average Neighbor Degree
# Works on G
an_degree = nx.algorithms.assortativity.average_neighbor_degree(G)
nx.set_node_attributes(G, an_degree, 'AveNeighDegree')
len(an_degree)
#an_degree

2054

In [29]:
# Betweenness
# https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality

#PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')
#eigenvector_dict = nx.eigenvector_centrality(L) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
#nx.set_node_attributes(L, eigenvector_dict, 'eigenvector')
len(betweenness_dict)

2054

In [30]:
# Clustering Coefficient
J = nx.DiGraph(G)

# requires DiGraph (instead of MultiDiGraph)
cluster_coef = nx.algorithms.cluster.clustering(J)
nx.set_node_attributes(G, cluster_coef, 'ClusterCoef')
len(cluster_coef)

2054

In [31]:
# Best Partition
import community
# requires undirected graph
I = G.to_undirected()

best_partition = community.best_partition(I)  # compute communities
nx.set_node_attributes(G, best_partition, 'BestPartition')
len(best_partition)

2054

In [32]:
# Greedy Modularity Communities
# Now theres 33 communities from the Greedy_Mode -> is greedy_mod effected by attributes? 
#     -> Only change was adding the betweenness attribute?
#     -> difference between directed and undirected? (H) directed doesn't compute here...
#     -> trying greedy mod on G -> apply to I -> nope. I -> I works
#     -> trying DiGraph (J) -> apply to I
from networkx.algorithms import community
communities = community.greedy_modularity_communities(I)
print(f"The graph has {len(communities)} communities.")

modularity_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        modularity_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

print(len(modularity_dict))
# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(G, modularity_dict, 'Greedy_Mod')

The graph has 40 communities.
2054


In [33]:
# Label Propagation
# Trying H -> I  nope. Not implemented for directed type
# Trying I -> I
# Quick.
communities = community.label_propagation_communities(I)
# <generator object label_propagation_communities at 0x7f7858f98450>

label_prop_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        label_prop_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

        
print(len(label_prop_dict))
# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(G, label_prop_dict, 'label_prop')

2054


## New Ancestor Chain Eval
- Incorporate Clustering at each stage.

In [34]:
df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
df = pd.concat([df.drop(['properties'], axis=1), df['properties'].apply(pd.Series)], axis=1)
df = df.reset_index().rename(columns={"index" : "NodeID"})
# Don't know why but the best partition label isn't set in the node attributes..
#     it's working now. must have been an execution order issue.
#df['BestPartition'] = df['NodeID'].apply(lambda x: (best_partition[x]))
df.head(1)

,NodeID,_node_type,_node_class,_display,_color,Orig_Node_ID,DegreeCentrality,AveNeighDegree,betweenness,ClusterCoef,BestPartition,Greedy_Mod,label_prop,process_guid,host,process_id,user,process_image,process_image_path,command_line,hashes,process_path,file_path,file_name,full_path,extension,ip_address,mac
0,0,Process,SysMonProc,svchost.exe,#FF0000,-290913363542393754,0.00341,55.833333,0.000061,0.0,0,11,124,{47ab858c-cadc-5eac-2200-000000000400},SCRANTON.dmevals.local,1448.0,None,svchost.exe,C:\Windows\System32,None,{},C:\Windows\System32\svchost.exe,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
def new_eval_ancestor_chain(G, starting_guid):
    for (p, d) in G.nodes(data=True):
        if 'process_guid' in d['properties'].keys():
            if starting_guid in d['properties']['process_guid']:
                selected_node = p
                
    #ancestors = nx.ancestors(G, selected_node)
    malicious_ancestors = [selected_node]
    print("##########################################")
    print("Conduct Ancestor Chain Evaluation")
    print("##########################################")
    print("")
    for i in list([selected_node] + list(nx.ancestors(G, selected_node))):
        if(G.node[i]['_node_type'] == 'Process'):
            print("==========================================")
            print("Node " + str(i) + " Data:")
            print("==========================================")
            print(G.node[i])
            print("==========================================")
            H, malicious_root_ancestor, ancestors, descendants = get_ancestor_descendant_graph(G, i)
            df = get_graph_dataframe(G)
            
            print("==========================================")
            print("Node " + str(i) + " Ancestor/Descendant Chain:")
            print("==========================================")
            current_node_eval(df, malicious_root_ancestor, ancestors, descendants)
            
            print("==========================================")
            print("Node " + str(i) + " Community Analysis:")
            print("==========================================")
            clusterings = ['Greedy_Mod', 'BestPartition', 'label_prop']
            for j in clusterings:
                print(j)
                get_community_analysis(df, j)
            
            answer = None
            while answer not in ("yes", "no"):
                answer = input("Malicious? Enter yes or no: ")
                if answer == "yes":
                    # Do this.
                    malicious_ancestors.append(i)
                elif answer == "no":
                    # Do that.
                    return malicious_ancestors
                else:
                    print("Please enter yes or no.")
    return malicious_ancestors

In [36]:
def get_ancestor_descendant_graph(G, malicious_root_ancestor):
    print(malicious_root_ancestor)

    descendants = nx.descendants(G, malicious_root_ancestor) #Returns all nodes reachable from source in G.
    print(len(descendants))

    ancestors = nx.ancestors(G, malicious_root_ancestor)
    print(len(ancestors))

    suspected_malicious = list(ancestors) + list(descendants)
    suspected_malicious.append(malicious_root_ancestor)
    print(len(suspected_malicious))

    H = G.subgraph(suspected_malicious)
    return H, malicious_root_ancestor, ancestors, descendants

In [37]:
def get_graph_dataframe(G):
    df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
    df = pd.concat([df.drop(['properties'], axis=1), df['properties'].apply(pd.Series)], axis=1)
    df = df.reset_index().rename(columns={"index" : "NodeID"})
    # Don't know why but the best partition label isn't set in the node attributes..
    #     it's working now. must have been an execution order issue.
    #df['BestPartition'] = df['NodeID'].apply(lambda x: (best_partition[x]))
    return df

In [40]:
def current_node_eval(df, malicious_root_ancestor, ancestors, descendants):
    df['Chain_Loc'] = np.nan
    df['Chain_Loc'].loc[df.NodeID.isin(list(ancestors))] = '1_Ancestor' #[['_display', 'DegreeCentrality']]
    df['Chain_Loc'].loc[df.NodeID.isin([malicious_root_ancestor])] = '2_Root' #[['_display', 'DegreeCentrality']]
    df['Chain_Loc'].loc[df.NodeID.isin(list(descendants))] = '3_Descendant' #[['_display', 'DegreeCentrality']]
    df['Chain_Loc_Int'] = np.nan
    df['Chain_Loc_Int'].loc[df.NodeID.isin([malicious_root_ancestor])] = 0 #[['_display', 'DegreeCentrality']]
    counter = 1
    for i in ancestors:
        df['Chain_Loc_Int'].loc[df['NodeID'] == i] = counter #[['_display', 'DegreeCentrality']]
        counter += 1
    columnList = ['_display', '_node_type', 'Chain_Loc', 'BestPartition', 'Greedy_Mod', 'label_prop', 'betweenness', 'DegreeCentrality', 'ClusterCoef', 'AveNeighDegree']
    display(df.loc[(((df['Chain_Loc']=='1_Ancestor') & (df['_node_type']=='Process'))| (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc'], ascending=True)[columnList])

In [41]:
new_eval_ancestor_chain(G, '{47ab858c-e13c-5eac-a903-000000000400}')

##########################################
Conduct Ancestor Chain Evaluation
##########################################

Node 1365 Data:
{'properties': {'process_guid': '{47ab858c-e13c-5eac-a903-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 8524, 'user': nan, 'process_image': 'â€®cod.3aka3.scr', 'process_image_path': 'C:\\ProgramData\\victim', 'command_line': '"C:\\ProgramData\\victim\\â€®cod.3aka3.scr" /S', 'hashes': {'sha1': '4B7FA56A4E85F88B98D11A6E018698AE3FBA5E62', 'md5': '9D1C5EF38E6073661C74660B3A71A76E', 'sha256': '0DF38A55D940F498478EB03683C94D4584236E100125B526A67650BA54DF4AE4', 'imphash': 'F00447512A354E59D39D2818AABA4A17'}, 'process_path': 'C:\\ProgramData\\victim\\â€®cod.3aka3.scr'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'â€®cod.3aka3.scr', '_color': '#FF0000', 'Orig_Node_ID': 691507765666240315, 'DegreeCentrality': 0.003896736483195324, 'AveNeighDegree': 1.0, 'betweenness': 0.0004643041277491504, 'ClusterCoef': 0, 'BestPartiti

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,_display,_node_type,Chain_Loc,BestPartition,Greedy_Mod,label_prop,betweenness,DegreeCentrality,ClusterCoef,AveNeighDegree
1461,Explorer.EXE,Process,1_Ancestor,13,9,187,2.364248e-04,0.005358,0.000000,0.666667
1365,â€®cod.3aka3.scr,Process,2_Root,11,4,116,4.643041e-04,0.003897,0.000000,1.000000
1250,cvtres.exe,Process,3_Descendant,23,5,72,2.373743e-07,0.001461,0.166667,0.000000
1076,control.exe,Process,3_Descendant,11,5,16,1.108063e-03,0.001461,0.000000,19.000000
1075,sdclt.exe,Process,3_Descendant,11,4,17,0.000000e+00,0.000974,0.000000,0.000000
1074,sdclt.exe,Process,3_Descendant,11,4,17,0.000000e+00,0.000974,0.000000,0.000000
1073,sdclt.exe,Process,3_Descendant,11,4,17,8.877799e-04,0.001461,0.000000,1.000000
998,cmd.exe,Process,3_Descendant,11,4,141,9.020223e-06,0.001461,0.000000,6.000000
690,sdelete64.exe,Process,3_Descendant,17,0,123,0.000000e+00,0.000974,0.000000,0.000000
919,csc.exe,Process,3_Descendant,23,5,71,1.091922e-05,0.002923,0.066667,0.250000


Node 1365 Community Analysis:
Greedy_Mod


NameError: name 'get_community_analysis' is not defined

In [54]:
new_eval_ancestor_chain(G, '{47ab858c-e558-5eac-f503-000000000400}')

##########################################
Conduct Ancestor Chain Evaluation
##########################################

Node 838 Data:
{'properties': {'process_guid': '{47ab858c-e558-5eac-f503-000000000400}', 'host': 'SCRANTON.dmevals.local', 'process_id': 1412, 'user': 'DMEVALS\\pbeesly', 'process_image': 'PsExec64.exe', 'process_image_path': 'C:\\Program Files\\SysinternalsSuite', 'command_line': '"C:\\Program Files\\SysinternalsSuite\\PsExec64.exe" -accepteula \\\\NASHUA -u dmevals\\pbeesly -p Fl0nk3rt0n!T0by -i 2 C:\\Windows\\Temp\\python.exe', 'hashes': {'sha1': 'FB0A150601470195C47B4E8D87FCB3F50292BEB2', 'md5': '9321C107D1F7E336CDA550A2BF049108', 'sha256': 'AD6B98C01EE849874E4B4502C3D7853196F6044240D3271E4AB3FC6E3C08E9A4', 'imphash': '159D56D406180A332FBC99290F30700E'}, 'process_path': 'C:\\Program Files\\SysinternalsSuite\\PsExec64.exe'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'PsExec64.exe', '_color': '#FF0000', 'Orig_Node_ID': -362988825189559677, '

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,_display,_node_type,Chain_Loc,Chain_Loc_Int,BestPartition,Greedy_Mod,label_prop,betweenness,DegreeCentrality,ClusterCoef,AveNeighDegree
838,PsExec64.exe,Process,2_Root,0.0,8,5,119,5.934357e-08,0.001461,0.166667,0.000000
1365,â€®cod.3aka3.scr,Process,1_Ancestor,3.0,11,4,117,4.643041e-04,0.003897,0.000000,1.000000
280,powershell.exe,Process,1_Ancestor,6.0,8,5,100,4.267990e-04,0.063322,0.000621,0.125984
281,PowerShell.exe,Process,1_Ancestor,7.0,11,5,101,1.331907e-03,0.010716,0.007143,24.000000
1053,sdclt.exe,Process,1_Ancestor,8.0,11,4,13,8.877799e-04,0.001461,0.000000,1.000000
1056,control.exe,Process,1_Ancestor,9.0,11,5,12,1.108063e-03,0.001461,0.000000,19.000000
993,cmd.exe,Process,1_Ancestor,10.0,11,4,13,6.791279e-04,0.003410,0.000000,1.400000
1461,Explorer.EXE,Process,1_Ancestor,16.0,13,9,187,2.364248e-04,0.005358,0.000000,0.666667


Node 838 Community Analysis:
Greedy_Mod


,in_all,out_all,all_all,in_proc,out_proc,all_proc,All_Type_Percent_Malicious,Proc_Type_Percent_Malicious
5,9.0,129.0,138.0,4.0,10.0,14.0,0.065217,0.285714
9,3.0,53.0,56.0,1.0,25.0,26.0,0.053571,0.038462
4,7.0,167.0,174.0,3.0,108.0,111.0,0.040230,0.027027


...

,in_all,out_all,all_all,in_proc,out_proc,all_proc,All_Type_Percent_Malicious,Proc_Type_Percent_Malicious
17,324.0,3.0,327.0,5.0,0.0,5.0,0.990826,1.000000
23,10.0,2.0,12.0,4.0,0.0,4.0,0.833333,1.000000
8,105.0,122.0,227.0,5.0,13.0,18.0,0.462555,0.277778
11,49.0,141.0,190.0,20.0,94.0,114.0,0.257895,0.175439
13,11.0,45.0,56.0,2.0,24.0,26.0,0.196429,0.076923
14,1.0,52.0,53.0,0.0,28.0,28.0,0.018868,0.000000
4,1.0,17.0,18.0,0.0,8.0,8.0,0.055556,0.000000


label_prop


,in_all,out_all,all_all,in_proc,out_proc,all_proc,All_Type_Percent_Malicious,Proc_Type_Percent_Malicious
124,323.0,2.0,325.0,5.0,0.0,5.0,0.993846,1.000000
106,5.0,0.0,5.0,2.0,0.0,2.0,1.000000,1.000000
76,1.0,1.0,2.0,1.0,0.0,1.0,0.500000,1.000000
55,2.0,2.0,4.0,1.0,0.0,1.0,0.500000,1.000000
70,2.0,1.0,3.0,1.0,0.0,1.0,0.666667,1.000000
69,3.0,1.0,4.0,1.0,0.0,1.0,0.750000,1.000000
135,4.0,0.0,4.0,2.0,0.0,2.0,1.000000,1.000000
100,100.0,0.0,100.0,2.0,0.0,2.0,1.000000,1.000000
13,4.0,1.0,5.0,4.0,0.0,4.0,0.800000,1.000000
130,5.0,0.0,5.0,2.0,0.0,2.0,1.000000,1.000000


Malicious? Enter yes or no:  no


[838, 838, 1365, 280, 281, 1053, 1056, 993]

## DataFrame Analysis

In [37]:
df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
df = pd.concat([df.drop(['properties'], axis=1), df['properties'].apply(pd.Series)], axis=1)
df = df.reset_index().rename(columns={"index" : "NodeID"})
# Don't know why but the best partition label isn't set in the node attributes..
#     it's working now. must have been an execution order issue.
#df['BestPartition'] = df['NodeID'].apply(lambda x: (best_partition[x]))
df.head(1)

,NodeID,_node_type,_node_class,_display,_color,Orig_Node_ID,DegreeCentrality,AveNeighDegree,betweenness,ClusterCoef,BestPartition,Greedy_Mod,label_prop,process_guid,host,process_id,user,process_image,process_image_path,command_line,hashes,process_path,file_path,file_name,full_path,extension,ip_address,mac
0,0,Process,SysMonProc,svchost.exe,#FF0000,-7312779468266332275,0.00341,55.833333,0.000061,0.0,0,11,124,{47ab858c-cadc-5eac-2200-000000000400},SCRANTON.dmevals.local,1448.0,None,svchost.exe,C:\Windows\System32,None,{},C:\Windows\System32\svchost.exe,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df['Chain_Loc'] = np.nan
df['Chain_Loc'].loc[df.NodeID.isin(list(ancestors))] = '1_Ancestor' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin([malicious_root_ancestor])] = '2_Root' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin(list(descendants))] = '3_Descendant' #[['_display', 'DegreeCentrality']]
df['Chain_Loc_Int'] = np.nan
df['Chain_Loc_Int'].loc[df.NodeID.isin([malicious_root_ancestor])] = 0 #[['_display', 'DegreeCentrality']]
counter = 1
for i in ancestors:
    df['Chain_Loc_Int'].loc[df['NodeID'] == i] = counter #[['_display', 'DegreeCentrality']]
    counter += 1
columnList = ['_display', '_node_type', 'Chain_Loc', 'Chain_Loc_Int', 'BestPartition', 'Greedy_Mod', 'label_prop', 'betweenness', 'DegreeCentrality', 'ClusterCoef', 'AveNeighDegree']
df.loc[(((df['Chain_Loc']=='1_Ancestor') & (df['_node_type']=='Process'))| (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc_Int'], ascending=True)[columnList]

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,_display,_node_type,Chain_Loc,Chain_Loc_Int,BestPartition,Greedy_Mod,label_prop,betweenness,DegreeCentrality,ClusterCoef,AveNeighDegree
838,PsExec64.exe,Process,2_Root,0.0,8,5,119,5.934357e-08,0.001461,0.166667,0.000000
1409,control.exe,File,1_Ancestor,1.0,11,5,12,0.000000e+00,0.000487,0.000000,1.000000
1998,PowerShell.exe,File,1_Ancestor,2.0,11,5,101,0.000000e+00,0.000487,0.000000,19.000000
1365,â€®cod.3aka3.scr,Process,1_Ancestor,3.0,11,4,117,4.643041e-04,0.003897,0.000000,1.000000
278,powershell.exe,File,1_Ancestor,4.0,11,4,142,0.000000e+00,0.005845,0.037879,42.250000
1367,â€®cod.3aka3.scr,File,1_Ancestor,5.0,11,4,117,0.000000e+00,0.000487,0.000000,6.000000
280,powershell.exe,Process,1_Ancestor,6.0,8,5,100,4.267990e-04,0.063322,0.000621,0.125984
281,PowerShell.exe,Process,1_Ancestor,7.0,11,5,101,1.331907e-03,0.010716,0.007143,24.000000
1053,sdclt.exe,Process,1_Ancestor,8.0,11,4,13,8.877799e-04,0.001461,0.000000,1.000000
1056,control.exe,Process,1_Ancestor,9.0,11,5,12,1.108063e-03,0.001461,0.000000,19.000000


In [73]:
df.head(1)

,NodeID,_node_type,_node_class,_display,_color,Orig_Node_ID,DegreeCentrality,AveNeighDegree,betweenness,ClusterCoef,BestPartition,Greedy_Mod,label_prop,host,process_id,user,process_image,process_image_path,command_line,hashes,process_path,file_path,file_name,full_path,extension,process_guid,ip_address,mac,Chain_Loc,Chain_Loc_Int
0,0,Process,Process,UsoClient.exe,#FF0000,-7627348004776576771,0.000753,0.0,0.0,0.0,0,13,0,NEWYORK.dmevals.local,1784.0,NEWYORK$,UsoClient.exe,C:\Windows\System32,C:\windows\system32\usoclient.exe StartScan,{},C:\Windows\System32\UsoClient.exe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Eval Communities

In [51]:
def get_community_analysis(df, clustering):
    df['Chain_Loc'] = df.Chain_Loc.fillna(False)
    #df.groupby('Chain_Loc', 'Greedy_Mod').count()
    Greedy_Mod_In = dict(df.loc[df.Chain_Loc!=False][clustering].value_counts())
    Greedy_Mod_Out = dict(df.loc[df.Chain_Loc==False][clustering].value_counts())
    Greedy_Mod_All = dict(df[clustering].value_counts())

    Greedy_Mod_In_Proc = dict(df.loc[(df.Chain_Loc!=False) & (df._node_type=='Process')][clustering].value_counts())
    Greedy_Mod_In_Proc

    Greedy_Mod_Out_Proc = dict(df.loc[(df.Chain_Loc==False) & (df._node_type=='Process')][clustering].value_counts())
    Greedy_Mod_All_Proc = dict(df.loc[(df._node_type=='Process')][clustering].value_counts())

    combo_dict = {}
    for i, j in Greedy_Mod_In.items():
        combo_dict[i] = {
            'in_all' : j,
            'out_all' : 0,
            'all_all' : Greedy_Mod_All[i],
            'in_proc' : 0,
            'out_proc' : 0,
            'all_proc' : 0,
        }
        if i in Greedy_Mod_Out:
            combo_dict[i]['out_all'] = Greedy_Mod_Out[i]
        combo_dict[i]['All_Type_Percent_Malicious'] = (combo_dict[i]['in_all'] / combo_dict[i]['all_all'])

        if i in Greedy_Mod_Out_Proc:
            combo_dict[i]['out_proc'] = Greedy_Mod_Out_Proc[i]

        if i in Greedy_Mod_In_Proc:
            combo_dict[i]['in_proc'] = Greedy_Mod_In_Proc[i]

        if i in Greedy_Mod_All_Proc:
            combo_dict[i]['all_proc'] = Greedy_Mod_All_Proc[i]
            combo_dict[i]['Proc_Type_Percent_Malicious'] = (combo_dict[i]['in_proc'] / combo_dict[i]['all_proc'])

    display(pd.DataFrame(combo_dict).transpose().sort_values('Proc_Type_Percent_Malicious', ascending=False))

In [40]:
#clustering = 'Greedy_Mod'
get_community_analysis(df, 'Greedy_Mod')

,in_all,out_all,all_all,in_proc,out_proc,all_proc,All_Type_Percent_Malicious,Proc_Type_Percent_Malicious
5,9.0,129.0,138.0,4.0,10.0,14.0,0.065217,0.285714
9,3.0,53.0,56.0,1.0,25.0,26.0,0.053571,0.038462
4,7.0,167.0,174.0,3.0,108.0,111.0,0.040230,0.027027


# Start at the bottom test - Nashua

In [98]:
df.host.value_counts()

SCRANTON.dmevals.local    508
NASHUA.dmevals.local      123
NEWYORK.dmevals.local      53
UTICA.dmevals.local        35
Name: host, dtype: int64

In [102]:
df.loc[((df['host']=='NASHUA.dmevals.local') & (df['_display'] == 'PSEXESVC.exe'))]

,NodeID,_node_type,_node_class,_display,_color,Orig_Node_ID,DegreeCentrality,AveNeighDegree,betweenness,ClusterCoef,BestPartition,Greedy_Mod,label_prop,process_guid,host,process_id,user,process_image,process_image_path,command_line,hashes,process_path,file_path,file_name,full_path,extension,ip_address,mac,Chain_Loc,Chain_Loc_Int
151,151,Process,SysMonProc,PSEXESVC.exe,#FF0000,-20068069117770139,0.001461,1.000000,0.000001,0.0,9,7,113,{5aa8ec29-e56d-5eac-6d03-000000000400},NASHUA.dmevals.local,8308.0,NT AUTHORITY\SYSTEM,PSEXESVC.exe,C:\Windows,C:\windows\PSEXESVC.exe,"{'sha1': 'F1E36E0E34276A5015040780E14B58EFD1112B76', 'md5': 'AE5BB9F3FFF1AEAAAD619BAB105B2391', 'sha256': '224F549F33854ED53667055786DC1073E64B7428FAE26F27DAB9828ED502BB99', 'imphash': '09D5553D2AA2F39BDE811B88883DE7D5'}",C:\Windows\PSEXESVC.exe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,152,Process,SysMonProc,PSEXESVC.exe,#FF0000,7981502332299158110,0.001461,1.000000,0.000001,0.0,9,7,110,{5aa8ec29-e52e-5eac-6a03-000000000400},NASHUA.dmevals.local,5920.0,NT AUTHORITY\SYSTEM,PSEXESVC.exe,C:\Windows,C:\windows\PSEXESVC.exe,"{'sha1': 'F1E36E0E34276A5015040780E14B58EFD1112B76', 'md5': 'AE5BB9F3FFF1AEAAAD619BAB105B2391', 'sha256': '224F549F33854ED53667055786DC1073E64B7428FAE26F27DAB9828ED502BB99', 'imphash': '09D5553D2AA2F39BDE811B88883DE7D5'}",C:\Windows\PSEXESVC.exe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,154,Process,SysMonProc,PSEXESVC.exe,#FF0000,-3057332116876962313,0.002923,0.333333,0.000004,0.0,9,7,49,{5aa8ec29-e4eb-5eac-6703-000000000400},NASHUA.dmevals.local,9204.0,NaN,PSEXESVC.exe,C:\windows,C:\windows\PSEXESVC.exe,"{'sha1': 'F1E36E0E34276A5015040780E14B58EFD1112B76', 'md5': 'AE5BB9F3FFF1AEAAAD619BAB105B2391', 'sha256': '224F549F33854ED53667055786DC1073E64B7428FAE26F27DAB9828ED502BB99', 'imphash': '09D5553D2AA2F39BDE811B88883DE7D5'}",C:\windows\PSEXESVC.exe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,156,Process,SysMonProc,PSEXESVC.exe,#FF0000,4131373192010027577,0.001461,19.000000,0.000028,0.0,9,7,51,{5aa8ec29-e5b7-5eac-7603-000000000400},NASHUA.dmevals.local,1624.0,NT AUTHORITY\SYSTEM,PSEXESVC.exe,C:\Windows,C:\windows\PSEXESVC.exe,"{'sha1': 'F1E36E0E34276A5015040780E14B58EFD1112B76', 'md5': 'AE5BB9F3FFF1AEAAAD619BAB105B2391', 'sha256': '224F549F33854ED53667055786DC1073E64B7428FAE26F27DAB9828ED502BB99', 'imphash': '09D5553D2AA2F39BDE811B88883DE7D5'}",C:\Windows\PSEXESVC.exe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,159,File,File,PSEXESVC.exe,#3CB371,7547748927891737162,0.001948,1.500000,0.000000,0.0,9,7,51,NaN,NASHUA.dmevals.local,NaN,NaN,NaN,NaN,NaN,"{'sha1': 'F1E36E0E34276A5015040780E14B58EFD1112B76', 'md5': 'AE5BB9F3FFF1AEAAAD619BAB105B2391', 'sha256': '224F549F33854ED53667055786DC1073E64B7428FAE26F27DAB9828ED502BB99', 'imphash': '09D5553D2AA2F39BDE811B88883DE7D5'}",NaN,C:\Windows,PSEXESVC.exe,C:\Windows\PSEXESVC.exe,NaN,NaN,NaN,NaN,NaN
2026,2026,File,File,PSEXESVC.exe,#3CB371,-7475341486478402990,0.000487,3.000000,0.000000,0.0,9,7,49,NaN,NASHUA.dmevals.local,NaN,NaN,NaN,NaN,NaN,{},NaN,C:\windows,PSEXESVC.exe,C:\windows\PSEXESVC.exe,NaN,NaN,NaN,NaN,NaN


In [110]:
# {5aa8ec29-e5b7-5eac-7603-000000000400}
M2, malicious_root_ancestor, ancestors, descendants, file_nodes, proc_nodes, ip_nodes = manual_recurse_pipeline(G, '{5aa8ec29-e655-5eac-8303-000000000400}')

Node Data:
{'properties': {'process_guid': '{5aa8ec29-e618-5eac-7e03-000000000400}', 'host': 'NASHUA.dmevals.local', 'process_id': 8772, 'user': 'DMEVALS\\pbeesly', 'process_image': 'cmd.exe', 'process_image_path': 'C:\\Windows\\System32', 'command_line': 'C:\\windows\\system32\\cmd.exe', 'hashes': {'sha1': '8DCA9749CD48D286950E7A9FA1088C937CBCCAD4', 'md5': 'D7AB69FAD18D4A643D84A271DFC0DBDF', 'sha256': 'FF79D3C4A0B7EB191783C323AB8363EBD1FD10BE58D8BCC96B07067743CA81D5', 'imphash': '272245E2988E1E430500B852C4FB5E18'}, 'process_path': 'C:\\Windows\\System32\\cmd.exe'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'cmd.exe', '_color': '#FF0000', 'Orig_Node_ID': 161627779468031659, 'DegreeCentrality': 0.0024354603019970775, 'AveNeighDegree': 0.0, 'betweenness': 5.696983162566263e-06, 'ClusterCoef': 0, 'BestPartition': 9, 'Greedy_Mod': 7, 'label_prop': 163}


Malicious? Enter yes or no:  no



##########################################
Confirm this as the senior malicious node?
##########################################
Node Data:
{'properties': {'process_guid': '{5aa8ec29-e655-5eac-8303-000000000400}', 'host': 'NASHUA.dmevals.local', 'process_id': 6640, 'user': 'DMEVALS\\pbeesly', 'process_image': 'sdelete64.exe', 'process_image_path': 'C:\\Windows\\Temp', 'command_line': '.\\sdelete64.exe  /accepteula "C:\\Users\\pbeesly\\Desktop\\working.zip"', 'hashes': {'sha1': '7BCD946326B67F806B3DB4595EDE9FBDF29D0C36', 'md5': '2B5CB081721B8BA454713119BE062491', 'sha256': 'FEEC1457836A5F84291215A2A003FCDE674E7E422DF8C4ED6FE5BB3B679CDC87', 'imphash': '342934F7499D0F57D88D4434E41B7BF9'}, 'process_path': 'C:\\Windows\\Temp\\sdelete64.exe'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'sdelete64.exe', '_color': '#FF0000', 'Orig_Node_ID': -1234035203258186817, 'DegreeCentrality': 0.000974184120798831, 'AveNeighDegree': 0.0, 'betweenness': 0.0, 'ClusterCoef': 0, 'BestP

Malicious? Enter yes or no:  yes


161
161
0
10
11


In [111]:
df['Chain_Loc'] = np.nan
df['Chain_Loc'].loc[df.NodeID.isin(list(ancestors))] = '1_Ancestor' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin([malicious_root_ancestor])] = '2_Root' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin(list(descendants))] = '3_Descendant' #[['_display', 'DegreeCentrality']]
df['Chain_Loc_Int'] = np.nan
df['Chain_Loc_Int'].loc[df.NodeID.isin([malicious_root_ancestor])] = 0 #[['_display', 'DegreeCentrality']]
counter = 1
for i in ancestors:
    df['Chain_Loc_Int'].loc[df['NodeID'] == i] = counter #[['_display', 'DegreeCentrality']]
    counter += 1
columnList = ['_display', '_node_type', 'process_guid', 'command_line', 'Chain_Loc', 'Chain_Loc_Int', 'BestPartition', 'Greedy_Mod', 'label_prop', 'betweenness', 'DegreeCentrality', 'ClusterCoef', 'AveNeighDegree']
df.loc[((df['Chain_Loc']=='1_Ancestor') | (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc_Int'], ascending=True)[columnList]

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,_display,_node_type,process_guid,command_line,Chain_Loc,Chain_Loc_Int,BestPartition,Greedy_Mod,label_prop,betweenness,DegreeCentrality,ClusterCoef,AveNeighDegree
161,sdelete64.exe,Process,{5aa8ec29-e655-5eac-8303-000000000400},".\sdelete64.exe /accepteula ""C:\Users\pbeesly\Desktop\working.zip""",2_Root,0.0,9,7,163,0.000000,0.000974,0.000000,0.000000
160,cmd.exe,Process,{5aa8ec29-e618-5eac-7e03-000000000400},C:\windows\system32\cmd.exe,1_Ancestor,1.0,9,7,163,0.000006,0.002435,0.000000,0.000000
960,python.exe,Process,{5aa8ec29-e5b7-5eac-7703-000000000400},"""C:\Windows\Temp\python.exe""",1_Ancestor,2.0,9,7,165,0.000040,0.010229,0.002381,18.526316
164,cmd.exe,File,NaN,NaN,1_Ancestor,3.0,9,7,166,0.000000,0.000974,0.000000,2.000000
166,sdelete64.exe,File,NaN,NaN,1_Ancestor,4.0,9,7,163,0.000000,0.001461,0.000000,0.000000
978,python.exe,Process,{5aa8ec29-e5b8-5eac-7903-000000000400},"""C:\Windows\Temp\python.exe""",1_Ancestor,5.0,4,32,163,0.000027,0.172431,0.023810,0.036932
980,python.exe,File,NaN,NaN,1_Ancestor,6.0,9,7,165,0.000000,0.002435,0.050000,74.800000
150,services.exe,Process,{5aa8ec29-cad7-5eac-0b00-000000000400},None,1_Ancestor,7.0,9,7,52,0.000018,0.004871,0.000000,2.888889
156,PSEXESVC.exe,Process,{5aa8ec29-e5b7-5eac-7603-000000000400},C:\windows\PSEXESVC.exe,1_Ancestor,8.0,9,7,51,0.000028,0.001461,0.000000,19.000000
157,services.exe,File,NaN,NaN,1_Ancestor,9.0,9,7,52,0.000000,0.000487,0.000000,9.000000


In [38]:
# Check the 'maliciousness' of the services.exe process. {5aa8ec29-cad7-5eac-0b00-000000000400}
# Now try landing lower - PSEXECSVC.exe -> remove the 'services' benign activity from the descendant chain. {5aa8ec29-e5b7-5eac-7603-000000000400}
M2, malicious_root_ancestor, ancestors, descendants, file_nodes, proc_nodes, ip_nodes = manual_recurse_pipeline(G, '{5aa8ec29-e5b7-5eac-7603-000000000400}')

Node Data:
{'properties': {'process_guid': '{5aa8ec29-cad7-5eac-0b00-000000000400}', 'host': 'NASHUA.dmevals.local', 'process_id': 720, 'user': None, 'process_image': 'services.exe', 'process_image_path': 'C:\\Windows\\System32', 'command_line': None, 'hashes': {}, 'process_path': 'C:\\Windows\\System32\\services.exe'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'services.exe', '_color': '#FF0000', 'Orig_Node_ID': -6077440615605945838, 'DegreeCentrality': 0.004870920603994155, 'AveNeighDegree': 2.888888888888889, 'betweenness': 1.780307238301957e-05, 'ClusterCoef': 0, 'BestPartition': 9, 'Greedy_Mod': 7, 'label_prop': 52}


Malicious? Enter yes or no:  no



##########################################
Confirm this as the senior malicious node?
##########################################
Node Data:
{'properties': {'process_guid': '{5aa8ec29-e5b7-5eac-7603-000000000400}', 'host': 'NASHUA.dmevals.local', 'process_id': 1624, 'user': 'NT AUTHORITY\\SYSTEM', 'process_image': 'PSEXESVC.exe', 'process_image_path': 'C:\\Windows', 'command_line': 'C:\\windows\\PSEXESVC.exe', 'hashes': {'sha1': 'F1E36E0E34276A5015040780E14B58EFD1112B76', 'md5': 'AE5BB9F3FFF1AEAAAD619BAB105B2391', 'sha256': '224F549F33854ED53667055786DC1073E64B7428FAE26F27DAB9828ED502BB99', 'imphash': '09D5553D2AA2F39BDE811B88883DE7D5'}, 'process_path': 'C:\\Windows\\PSEXESVC.exe'}, '_node_type': 'Process', '_node_class': 'SysMonProc', '_display': 'PSEXESVC.exe', '_color': '#FF0000', 'Orig_Node_ID': -4595658238231141275, 'DegreeCentrality': 0.0014612761811982466, 'AveNeighDegree': 19.0, 'betweenness': 2.777279291751053e-05, 'ClusterCoef': 0, 'BestPartition': 9, 'Greedy_Mod': 7, 'label_

Malicious? Enter yes or no:  yes


156
156
39
3
43


In [39]:
df['Chain_Loc'] = np.nan
df['Chain_Loc'].loc[df.NodeID.isin(list(ancestors))] = '1_Ancestor' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin([malicious_root_ancestor])] = '2_Root' #[['_display', 'DegreeCentrality']]
df['Chain_Loc'].loc[df.NodeID.isin(list(descendants))] = '3_Descendant' #[['_display', 'DegreeCentrality']]
df['Chain_Loc_Int'] = np.nan
df['Chain_Loc_Int'].loc[df.NodeID.isin([malicious_root_ancestor])] = 0 #[['_display', 'DegreeCentrality']]
counter = 1
for i in ancestors:
    df['Chain_Loc_Int'].loc[df['NodeID'] == i] = counter #[['_display', 'DegreeCentrality']]
    counter += 1
columnList = ['_display', '_node_type', 'command_line', 'Chain_Loc', 'Chain_Loc_Int', 'BestPartition', 'Greedy_Mod', 'label_prop', 'betweenness', 'DegreeCentrality', 'ClusterCoef', 'AveNeighDegree']
df.loc[((df['Chain_Loc']=='1_Ancestor') | (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc_Int'], ascending=True)[columnList]

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,_display,_node_type,command_line,Chain_Loc,Chain_Loc_Int,BestPartition,Greedy_Mod,label_prop,betweenness,DegreeCentrality,ClusterCoef,AveNeighDegree
156,PSEXESVC.exe,Process,C:\windows\PSEXESVC.exe,2_Root,0.0,9,7,51,0.000028,0.001461,0.000000,19.000000
157,services.exe,File,NaN,1_Ancestor,1.0,9,7,52,0.000000,0.000487,0.000000,9.000000
150,services.exe,Process,None,1_Ancestor,2.0,9,7,52,0.000018,0.004871,0.000000,2.888889
159,PSEXESVC.exe,File,NaN,1_Ancestor,3.0,9,7,51,0.000000,0.001948,0.000000,1.500000
160,cmd.exe,Process,C:\windows\system32\cmd.exe,3_Descendant,NaN,9,7,164,0.000006,0.002435,0.000000,0.000000
161,sdelete64.exe,Process,".\sdelete64.exe /accepteula ""C:\Users\pbeesly\Desktop\working.zip""",3_Descendant,NaN,9,7,164,0.000000,0.000974,0.000000,0.000000
162,sdelete64.exe,Process,".\sdelete64.exe /accepteula ""C:\Windows\Temp\Rar.exe""",3_Descendant,NaN,9,7,164,0.000000,0.000974,0.000000,0.000000
163,sdelete64.exe,Process,".\sdelete64.exe /accepteula ""C:\Users\pbeesly\AppData\Roaming\working.zip""",3_Descendant,NaN,9,7,164,0.000000,0.000974,0.000000,0.000000
963,python.exe,Process,"""C:\Windows\Temp\python.exe""",3_Descendant,NaN,9,7,168,0.000040,0.010229,0.002381,18.526316
981,python.exe,Process,"""C:\Windows\Temp\python.exe""",3_Descendant,NaN,4,32,164,0.000027,0.172431,0.023810,0.036932


In [42]:
columnList = ['_display', '_node_type', 'Chain_Loc', 'BestPartition', 'Greedy_Mod', 'label_prop', 'betweenness', 'DegreeCentrality', 'ClusterCoef', 'AveNeighDegree']
df.loc[((df['Chain_Loc']=='1_Ancestor') | (df['Chain_Loc']=='2_Root') | ((df['Chain_Loc']=='3_Descendant') & (df['_node_type']=='Process')))].sort_values(by=['Chain_Loc'], ascending=True)[columnList]

,_display,_node_type,Chain_Loc,BestPartition,Greedy_Mod,label_prop,betweenness,DegreeCentrality,ClusterCoef,AveNeighDegree
150,services.exe,Process,1_Ancestor,9,7,52,0.000018,0.004871,0.000000,2.888889
157,services.exe,File,1_Ancestor,9,7,52,0.000000,0.000487,0.000000,9.000000
159,PSEXESVC.exe,File,1_Ancestor,9,7,51,0.000000,0.001948,0.000000,1.500000
156,PSEXESVC.exe,Process,2_Root,9,7,51,0.000028,0.001461,0.000000,19.000000
160,cmd.exe,Process,3_Descendant,9,7,164,0.000006,0.002435,0.000000,0.000000
161,sdelete64.exe,Process,3_Descendant,9,7,164,0.000000,0.000974,0.000000,0.000000
162,sdelete64.exe,Process,3_Descendant,9,7,164,0.000000,0.000974,0.000000,0.000000
163,sdelete64.exe,Process,3_Descendant,9,7,164,0.000000,0.000974,0.000000,0.000000
963,python.exe,Process,3_Descendant,9,7,168,0.000040,0.010229,0.002381,18.526316
981,python.exe,Process,3_Descendant,4,32,164,0.000027,0.172431,0.023810,0.036932


In [172]:
df['Chain_Loc'] = df.Chain_Loc.fillna(False)
#df.groupby('Chain_Loc', 'Greedy_Mod').count()
Greedy_Mod_In = dict(df.loc[df.Chain_Loc!=False]['Greedy_Mod'].value_counts())
Greedy_Mod_Out = dict(df.loc[df.Chain_Loc==False]['Greedy_Mod'].value_counts())
Greedy_Mod_All = dict(df['Greedy_Mod'].value_counts())

Greedy_Mod_In_Proc = dict(df.loc[(df.Chain_Loc!=False) & (df._node_type=='Process')]['Greedy_Mod'].value_counts())
Greedy_Mod_In_Proc

Greedy_Mod_Out_Proc = dict(df.loc[(df.Chain_Loc==False) & (df._node_type=='Process')]['Greedy_Mod'].value_counts())
Greedy_Mod_All_Proc = dict(df.loc[(df._node_type=='Process')]['Greedy_Mod'].value_counts())

combo_dict = {}
for i, j in Greedy_Mod_In.items():
    combo_dict[i] = {
        'in_all' : j,
        'out_all' : 0,
        'all_all' : Greedy_Mod_All[i],
        'in_proc' : 0,
        'out_proc' : 0,
        'all_proc' : 0,
    }
    if i in Greedy_Mod_Out:
        combo_dict[i]['out_all'] = Greedy_Mod_Out[i]
    combo_dict[i]['All_Type_Percent_Malicious'] = (combo_dict[i]['in_all'] / combo_dict[i]['all_all'])
    
    if i in Greedy_Mod_Out_Proc:
        combo_dict[i]['out_proc'] = Greedy_Mod_Out_Proc[i]
    
    if i in Greedy_Mod_In_Proc:
        combo_dict[i]['in_proc'] = Greedy_Mod_In_Proc[i]

    if i in Greedy_Mod_All_Proc:
        combo_dict[i]['all_proc'] = Greedy_Mod_All_Proc[i]
        combo_dict[i]['Proc_Type_Percent_Malicious'] = (combo_dict[i]['in_proc'] / combo_dict[i]['all_proc'])
        
pd.DataFrame(combo_dict).transpose().sort_values('Proc_Type_Percent_Malicious', ascending=False)

,in_all,out_all,all_all,in_proc,out_proc,all_proc,All_Type_Percent_Malicious,Proc_Type_Percent_Malicious
19,11.0,3.0,14.0,2.0,0.0,2.0,0.785714,1.000000
32,3.0,0.0,3.0,1.0,0.0,1.0,1.000000,1.000000
7,38.0,28.0,66.0,19.0,14.0,33.0,0.575758,0.575758
12,6.0,30.0,36.0,2.0,6.0,8.0,0.166667,0.250000
3,15.0,193.0,208.0,2.0,119.0,121.0,0.072115,0.016529
20,1.0,12.0,13.0,0.0,8.0,8.0,0.076923,0.000000
17,1.0,15.0,16.0,0.0,8.0,8.0,0.062500,0.000000
4,1.0,173.0,174.0,0.0,111.0,111.0,0.005747,0.000000
38,1.0,0.0,1.0,0.0,0.0,0.0,1.000000,NaN


In [174]:
def get_community_analysis(df, clustering):
    df['Chain_Loc'] = df.Chain_Loc.fillna(False)
    #df.groupby('Chain_Loc', 'Greedy_Mod').count()
    Greedy_Mod_In = dict(df.loc[df.Chain_Loc!=False][clustering].value_counts())
    Greedy_Mod_Out = dict(df.loc[df.Chain_Loc==False][clustering].value_counts())
    Greedy_Mod_All = dict(df[clustering].value_counts())

    Greedy_Mod_In_Proc = dict(df.loc[(df.Chain_Loc!=False) & (df._node_type=='Process')][clustering].value_counts())
    Greedy_Mod_In_Proc

    Greedy_Mod_Out_Proc = dict(df.loc[(df.Chain_Loc==False) & (df._node_type=='Process')][clustering].value_counts())
    Greedy_Mod_All_Proc = dict(df.loc[(df._node_type=='Process')][clustering].value_counts())

    combo_dict = {}
    for i, j in Greedy_Mod_In.items():
        combo_dict[i] = {
            'in_all' : j,
            'out_all' : 0,
            'all_all' : Greedy_Mod_All[i],
            'in_proc' : 0,
            'out_proc' : 0,
            'all_proc' : 0,
        }
        if i in Greedy_Mod_Out:
            combo_dict[i]['out_all'] = Greedy_Mod_Out[i]
        combo_dict[i]['All_Type_Percent_Malicious'] = (combo_dict[i]['in_all'] / combo_dict[i]['all_all'])

        if i in Greedy_Mod_Out_Proc:
            combo_dict[i]['out_proc'] = Greedy_Mod_Out_Proc[i]

        if i in Greedy_Mod_In_Proc:
            combo_dict[i]['in_proc'] = Greedy_Mod_In_Proc[i]

        if i in Greedy_Mod_All_Proc:
            combo_dict[i]['all_proc'] = Greedy_Mod_All_Proc[i]
            combo_dict[i]['Proc_Type_Percent_Malicious'] = (combo_dict[i]['in_proc'] / combo_dict[i]['all_proc'])

    return pd.DataFrame(combo_dict).transpose().sort_values('Proc_Type_Percent_Malicious', ascending=False)

In [175]:
#clustering = 'Greedy_Mod'
get_community_analysis(df, 'Greedy_Mod')

,in_all,out_all,all_all,in_proc,out_proc,all_proc,All_Type_Percent_Malicious,Proc_Type_Percent_Malicious
19,11.0,3.0,14.0,2.0,0.0,2.0,0.785714,1.000000
32,3.0,0.0,3.0,1.0,0.0,1.0,1.000000,1.000000
7,38.0,28.0,66.0,19.0,14.0,33.0,0.575758,0.575758
12,6.0,30.0,36.0,2.0,6.0,8.0,0.166667,0.250000
3,15.0,193.0,208.0,2.0,119.0,121.0,0.072115,0.016529
20,1.0,12.0,13.0,0.0,8.0,8.0,0.076923,0.000000
17,1.0,15.0,16.0,0.0,8.0,8.0,0.062500,0.000000
4,1.0,173.0,174.0,0.0,111.0,111.0,0.005747,0.000000
38,1.0,0.0,1.0,0.0,0.0,0.0,1.000000,NaN


In [176]:
get_community_analysis(df, 'BestPartition')

,in_all,out_all,all_all,in_proc,out_proc,all_proc,All_Type_Percent_Malicious,Proc_Type_Percent_Malicious
9,39.0,29.0,68.0,19.0,14.0,33.0,0.573529,0.575758
10,6.0,28.0,34.0,2.0,5.0,7.0,0.176471,0.285714
4,4.0,14.0,18.0,1.0,7.0,8.0,0.222222,0.125000
8,11.0,216.0,227.0,2.0,16.0,18.0,0.048458,0.111111
2,15.0,165.0,180.0,2.0,112.0,114.0,0.083333,0.017544
14,1.0,52.0,53.0,0.0,28.0,28.0,0.018868,0.000000
11,1.0,189.0,190.0,0.0,114.0,114.0,0.005263,0.000000


In [177]:
get_community_analysis(df, 'label_prop')

,in_all,out_all,all_all,in_proc,out_proc,all_proc,All_Type_Percent_Malicious,Proc_Type_Percent_Malicious
165,18.0,1.0,19.0,1.0,0.0,1.0,0.947368,1.000000
52,2.0,0.0,2.0,1.0,0.0,1.0,1.000000,1.000000
51,1.0,1.0,2.0,1.0,0.0,1.0,0.500000,1.000000
50,1.0,1.0,2.0,1.0,0.0,1.0,0.500000,1.000000
48,1.0,1.0,2.0,1.0,0.0,1.0,0.500000,1.000000
33,2.0,0.0,2.0,1.0,0.0,1.0,1.000000,1.000000
145,2.0,0.0,2.0,2.0,0.0,2.0,1.000000,1.000000
49,2.0,1.0,3.0,1.0,0.0,1.0,0.666667,1.000000
66,2.0,1.0,3.0,1.0,0.0,1.0,0.666667,1.000000
113,3.0,0.0,3.0,3.0,0.0,3.0,1.000000,1.000000
